## Our clustering algorithm evaluation
Evaluating our clustering algorithm on bookswagon.com pages. The aim is to calculate precision and recall for "book details" cluster and the "catalog" cluster in bookswagon.com.

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import ast
FILEPATH = '../../../datasets/blackwells.csv'
FILEPATH

'../../../datasets/blackwells.csv'

In [2]:
df = pd.read_csv(FILEPATH, converters={'bitset': ast.literal_eval, 'tag_count': ast.literal_eval})

## Data analisys
Some preliminary analisys of the dataset

In [3]:
print("First 5 rows")
print("------------")
df.head()

First 5 rows
------------


url  \
0           https://blackwells.co.uk/bookshop/basket   
1          https://blackwells.co.uk/bookshop/search/   
2             https://blackwells.co.uk/bookshop/home   
3  https://blackwells.co.uk/bookshop/product/9781...   
4          https://blackwells.co.uk/bookshop/mapping   

                                referer_url  \
0    https://blackwells.co.uk/bookshop/home   
1    https://blackwells.co.uk/bookshop/home   
2    https://blackwells.co.uk/bookshop/home   
3    https://blackwells.co.uk/bookshop/home   
4  https://blackwells.co.uk/bookshop/basket   

                                                 src  \
0  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
1  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
2  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
3  \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
4  \n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" c...   

               shingle_vector    label  \
0    (0, 1, 5, 1, 1, 6, 3, 1)      NaN   
1    (0, 1, 5, 1, 1, 0, 3, 0)     list   
2    (0, 1, 0, 1, 0, 0, 3, 1)      NaN   
3    (0, 1, 1, 1, 1, 0, 0, 1)  product   
4  (2, 22, 1, 1, 7, 15, 7, 5)      NaN   

                                           tag_count  \
0  [0.0019569471624266144, 0.0019569471624266144,...   
1  [0.0012970168612191958, 0.0012970168612191958,...   
2  [0.0011655011655011655, 0.0011655011655011655,...   
3  [0.0008116883116883117, 0.0008116883116883117,...   
4  [0.008333333333333333, 0.008333333333333333, 0...   

                                              bitset  
0  [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...  
1  [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, ...  
2  [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...  
3  [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...

In [4]:
print("No. of rows and columns")
print("-----------------------")
df.shape

No. of rows and columns
-----------------------


(10919, 7)

In [5]:
print("Check null values")
print("-----------------")
df.isnull().any().any()

Check null values
-----------------


True

In [6]:
print("Check duplicate values")
print("----------------------")
len(df['url'].unique()) != df.shape[0]

Check duplicate values
----------------------


False

In [7]:
print("DataFrame column types")
print("----------------------")
df.info()

DataFrame column types
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10919 entries, 0 to 10918
Data columns (total 7 columns):
url               10919 non-null object
referer_url       10919 non-null object
src               10919 non-null object
shingle_vector    10919 non-null object
label             10899 non-null object
tag_count         10919 non-null object
bitset            10919 non-null object
dtypes: object(7)
memory usage: 597.2+ KB


In [8]:
print("Some stats")
print("----------------")
df[['url','referer_url','src','shingle_vector','label']].describe()

Some stats
----------------


url  \
count                                               10919   
unique                                              10919   
top     https://blackwells.co.uk/bookshop/product/Fool...   
freq                                                    1   

                                   referer_url  \
count                                    10919   
unique                                    6375   
top     https://blackwells.co.uk/bookshop/home   
freq                                        12   

                                                      src  \
count                                               10919   
unique                                              10525   
top     \n\n\n        \n<!DOCTYPE html>\n<html lang="e...   
freq                                                    7   

                  shingle_vector    label  
count                      10919    10899  
unique                        73        2  
top     (0, 1, 5, 0, 1, 0, 3, 0)  product  
freq                        2197    10405

In [9]:
fmt_string = 'There are {} row with {} label'
print(fmt_string.format(len(df[df['label'].isnull()]),'no'))
print(fmt_string.format(len(df[df['label']=='product']), 'product'))
print(fmt_string.format(len(df[df['label']=='list']), 'list'))

There are 20 row with no label
There are 10405 row with product label
There are 494 row with list label


## Run MeanShift clustering algorithm

In [10]:
#add top level folder to sys.path
import sys
sys.path.append('../../../')

In [11]:
from astarwars_clustering.clustering import clusteringevaluation
from astarwars_clustering.utils import utility
from astarwars_clustering.clustering.structural_clustering import dbscanclustering, meanshiftclustering

In [ ]:
clustering = meanshiftclustering(featurematrix=df['tag_count'].tolist())

In [ ]:
predictedLabels = clustering.labels_
noOfClusters = np.unique(predictedLabels)
df['predicted_label'] = predictedLabels
print('There are ' + str(noOfClusters) + 'clusters')
print()
print()
print('Cluster labels:')
noOfClusters

In [ ]:
cluster_fmt = 'cluster n. {} has {} pages'
noOfPages = 0

for index ,el in enumerate(noOfClusters):
    print(cluster_fmt.format(index ,utility.count_occurrences(predictedLabels,el)))

In [ ]:
df[df['predicted_label'] == 0]['url'].head(10)

In [ ]:
df[df['predicted_label'] == 1]['url'].head(10)

In [ ]:
df[df['predicted_label'] == 2]['url'].head(10)

## Evaluate recall and precision

In [ ]:
p1,r1=clusteringevaluation.calculate_precision_and_recall(df,clustering,'product',0)

In [ ]:
p1,r1=clusteringevaluation.calculate_precision_and_recall(df,clustering,'list',0)

# Clustering with DBSCAN algorithm

In [ ]:
dbscanclustering=dbscanclustering()